# Trigger Satellite Segmentation

Description: Triggers satellite segmentation by going through crop db entries, fetching satellite image, adding arrow and triggering satellite segmentation + field point prediction.

Author: Ivan Zvonkov

Date: Nov 10, 2023


In [4]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=RLFBNlVcbFdwB96PBh89C2LKDKv4I5&access_type=offline&code_challenge=cifg5cFMqKOkS_cPAFQIBIvYGMw_RIOmJVl-xjRWanE&code_challenge_method=S256


Credentials saved to file: [/Users/izvonkov/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "bsos-geog-harvest1" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [114]:
import geopandas as gpd
import pandas as pd
import sys

from google.cloud import firestore
from tqdm import tqdm
from pathlib import Path

sys.path.append("..")

from gcp.get_satellite_img.main import hello_world

In [6]:
df_all = pd.read_csv("../data/street2sat-v2.csv")

/var/folders/1v/87y9n_d5143c_6cp072v3b1c0000gn/T/ipykernel_5087/2016021947.py:1: DtypeWarning: Columns (9,12,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv("../data/street2sat-v2.csv")


In [13]:
df_crops = df_all[df_all["is_crop"] == True].copy()

In [14]:
input_img_str_split = df_crops["input_img"].str.split("/")
df_crops["country_folder"] = input_img_str_split.str[3]
df_crops["country_subfolder"] = input_img_str_split.str[4]

In [15]:
class MockRequest:
    def __init__(self, request):
        self.request_json = request
    
    def get_json(self, silent):
        return self.request_json

In [17]:
df_crops["country_folder"].value_counts()

country_folder
NIGERIA_v2             502615
ZAMBIA_v2              254999
Uganda_v2              252731
KENYA_v2               143288
USA_v2                  54253
TANZANIA_v2             11130
MPONGWE_v2               1021
test_set                  695
test_set_divided_v3       695
test_set_divided_v2       200
Name: count, dtype: int64

In [131]:
df_crops["input_img"].str.endswith(".THM").sum()

200

In [173]:
country_df = df_crops[df_crops["country_folder"] == "TANZANIA_v2"]

In [174]:
# Take only rows with no associated satellite image yet
country_df = country_df[country_df["mapbox_img_source"].isna() | country_df["bing_img_source"].isna()]

In [175]:
# 1282
# 1000 takes 30 mins
for i in tqdm(range(0, 2000)):
    if country_df.iloc[i]["input_img"].endswith(".THM"):
        continue
    resp = hello_world(MockRequest({"name": country_df.iloc[i]["name"]}))
    print(resp)

  0%|                                                  | 0/2000 [00:00<?, ?it/s]

Fetching from: https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/256/18/155274/133814?access_token=pk.eyJ1IjoiaXp2b25rb3YiLCJhIjoiY2t0YXU3MTE3MDJuNjJ1bW9xYzc3bTlrMCJ9.fiGLAZ29dmYv2sqlxuaB-A
100% [............................................................] 7916 / 7916Fetching from: http://ecn.t0.tiles.virtualearth.net/tiles/a300101313030221230.jpeg?g=14009
100% [..........................................................] 12356 / 12356

  0%|                                        | 2/2000 [00:06<1:40:15,  3.01s/it]

{'mapbox_fetched_img': True, 'mapbox_img': 'mapbox/155274/133814/18.jpg', 'mapbox_added_arrow': True, 'bing_fetched_img': True, 'bing_img': 'bing/155274/133814/18.jpg', 'bing_added_arrow': True}


  0%|                                        | 3/2000 [00:08<1:34:00,  2.82s/it]

...

{'mapbox_fetched_img': False, 'mapbox_img': 'mapbox/154692/133909/18.jpg', 'mapbox_added_arrow': True, 'bing_fetched_img': False, 'bing_img': 'bing/154692/133909/18.jpg', 'bing_added_arrow': True}


 64%|█████████████████████████              | 1286/2000 [56:30<25:34,  2.15s/it]

{'mapbox_fetched_img': False, 'mapbox_img': 'mapbox/154692/133909/18.jpg', 'mapbox_added_arrow': True, 'bing_fetched_img': False, 'bing_img': 'bing/154692/133909/18.jpg', 'bing_added_arrow': True}


 64%|█████████████████████████              | 1286/2000 [56:31<31:22,  2.64s/it]


KeyboardInterrupt: 

In [155]:
# TODO regenerate data/street2sat-v2.csv
db = firestore.Client()
coll = db.collection("street2sat-v2").where("is_crop", "==", True)

result = coll.count().get()
entry_count = result[0][0].value
print(f"As of {result[0][0].read_time} number of entries: {entry_count}")

/Users/izvonkov/nasaharvest/street2sat/venv/lib/python3.9/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


As of 2023-11-21 17:21:39.671336+00:00 number of entries: 1221627


In [156]:
total_runs = (entry_count // 1000) + 1
all_docs = []
docs = coll.order_by("input_img")
for i in tqdm(range(total_runs), desc="Queries"):
    for doc in docs.limit(1000).stream():
        all_docs.append(doc.to_dict())
    last_input_img = all_docs[-1]["input_img"]
    docs = coll.order_by("input_img").start_after({"input_img": last_input_img})

Queries: 100%|██████████████████████████████| 1222/1222 [17:38<00:00,  1.15it/s]


In [157]:
df = pd.DataFrame(all_docs)

In [158]:
df = df[~df["bing_field_coord"].isna() | ~df["mapbox_field_coord"].isna()]

In [159]:
df = df[~df["results"].isna()]
df = df[~df["coord"].isna()].copy().reset_index()

In [160]:
len(df)

9712

In [161]:
predictions = pd.json_normalize(df["results"])
crops = list(predictions.columns[1:])
df = pd.concat([df, predictions], axis=1)
df["dominant_crop"] = df[crops].apply(lambda x: max(dict(x), key=dict(x).get), axis=1)

In [162]:
input_img_str_split = df["input_img"].str.split("/")
df["country_folder"] = input_img_str_split.str[3]
df["country_subfolder"] = input_img_str_split.str[4]

In [163]:
df["road_lat"] = df["coord"].map(lambda coord: coord[0])
df["road_lon"] = df["coord"].map(lambda coord: coord[1])

In [164]:
df["country_folder"].value_counts()

country_folder
NIGERIA_v2    1983
Uganda_v2     1972
KENYA_v2      1968
USA_v2        1908
ZAMBIA_v2     1881
Name: count, dtype: int64

In [165]:
df.to_csv("../data/street2sat-v2-points-in-field.csv", index=False)

In [169]:
df["key"] = df["name"]
df["name"] = df["dominant_crop"]

In [170]:
def generate_gdf(cols, lon, lat):
    points = gpd.points_from_xy(df[lon], df[lat], crs="EPSG:4326")
    gdf = gpd.GeoDataFrame(df[cols], geometry=points)
    return gdf

road_cols = ["input_img", "name", "time"] + crops
bing_cols = ["input_img", "name", "time", "bing_img_source", "bing_img_segmentation"] + crops
mapbox_cols = ["input_img", "name", "time", "mapbox_img_source", "mapbox_img_segmentation"] + crops

gdf_road = generate_gdf(cols=road_cols, lat="road_lat", lon="road_lon")
gdf_bing = generate_gdf(cols=bing_cols, lat="bing_field_coord_lat", lon="bing_field_coord_lon")
gdf_mapbox = generate_gdf(cols=mapbox_cols, lat="mapbox_field_coord_lat", lon="mapbox_field_coord_lon")

In [171]:
Path("../data/points").mkdir(exist_ok=True)

In [172]:
gdf_road.to_file("../data/points/road.geojson", driver="GeoJSON")
gdf_bing.to_file("../data/points/field_bing.geojson", driver="GeoJSON")
gdf_mapbox.to_file("../data/points/field_mapbox.geojson", driver="GeoJSON")